In [42]:
import numpy as np
from pydgrid.pydgrid import grid
from pydgrid.plot_bokeh import plot_results
import sympy as sym
import pydae.build as db

In [43]:
data = {
        "buses":[
                 {"bus": "B1",  "pos_x":   0, "pos_y":   0, "units": "m", "U_kV":20.0},
                 {"bus": "B2",  "pos_x":  10, "pos_y":   0, "units": "m", "U_kV":0.4},
                 {"bus": "B3",  "pos_x": 100, "pos_y":  0, "units": "m", "U_kV":0.4}
                ],
        "grid_formers":[
                        {"bus": "B1",
                        "bus_nodes": [1, 2, 3], "deg": [0, -120, -240],
                        "kV": [11.547, 11.547, 11.547]}
                       ],
        "transformers":[
                        {"bus_j": "B1",  "bus_k": "B2",  "S_n_kVA": 1000.0, "U_j_kV":20, "U_k_kV":0.42,
                         "R_cc_pu": 0.01, "X_cc_pu":0.04, "connection": "Dyn11",   "conductors_j": 3, "conductors_k": 4},
                       ],
        "lines":[
                 {"bus_j": "B2",  "bus_k": "B3",  "code": "lv_cu_150", "m": 100.0},
                ],
        "loads":[
                 {"bus": "B3" , "kVA": 300.0, "pf": 0.85,"type":"3P+N"}
                ],
        "shunts":[
                 {"bus": "B2" , "R": 0.001, "X": 0.0, "bus_nodes": [4,0]},
                 {"bus": "B3" , "R": 0.01, "X": 0.0, "bus_nodes": [4,0]}
                 ],
        "line_codes":
            {"lv_cu_150":  {"Rph":0.167,"Xph":0.08, "Rn":0.167, "Xn": 0.08}
            }
       }

In [44]:
grid_1 = grid()
grid_1.read(data)  # Load data
grid_1.pf()  # solve power flow
p=plot_results(grid_1)

In [45]:
V_node = grid_1.V_node
I_node = grid_1.I_node
V_node_new = np.copy(V_node)

In [46]:
Y_ii = grid_1.Y_ii.toarray()
Y_iv = grid_1.Y_iv
inv_Y_ii = np.linalg.inv(Y_ii)
N_nz_nodes = grid_1.params_pf[0].N_nz_nodes
N_v = grid_1.params_pf[0].N_nodes_v

In [47]:
V_known   = np.copy(grid_1.V_node[0:N_v,:])
I_known = grid_1.I_node[N_v:,:]
I_aux = ( I_known - Y_iv @ V_known)  
V_unknown = inv_Y_ii[:,0:N_nz_nodes] @ I_aux[0:N_nz_nodes,:]
V_node_new[0:N_v,:] = V_known 
V_node_new[N_v:,:]  = V_unknown 

In [48]:
import sympy as sym

In [49]:
def grid2dae(nodes_list,V_node,I_node,Y_iv,inv_Y_ii,load_buses):
    
    N_v = Y_iv.shape[1]   # number of nodes with known voltages
    I_node_sym_list = []
    V_node_sym_list = []
    v_list = []
    i_list = []
    v_list_str = []
    i_list_str = []
    i_node = []
    v_num_list = []
    i_num_list = []
    n2a = {'1':'a','2':'b','3':'c','4':'n'}
    a2n = {'a':'1','b':'2','c':'3','n':'4'}

    # every voltage bus and current bus injection is generted as sympy symbol
    # the voltages ar named as v_{bus_name}_{n2a[phase]}_r
    # the currents ar named as i_{bus_name}_{n2a[phase]}_r
    inode = 0
    for node in grid_1.nodes:
        bus_name,phase = node.split('.')
        i_real = sym.Symbol(f"i_{bus_name}_{n2a[phase]}_r", real=True)
        i_imag = sym.Symbol(f"i_{bus_name}_{n2a[phase]}_i", real=True)
        v_real = sym.Symbol(f"v_{bus_name}_{n2a[phase]}_r", real=True)
        v_imag = sym.Symbol(f"v_{bus_name}_{n2a[phase]}_i", real=True)    

        v_list += [v_real,v_imag]  
        i_list += [i_real,i_imag]

        v_list_str += [str(v_real),str(v_imag)]
        i_list_str += [str(i_real),str(i_imag)]

        v_num_list += [V_node[inode].real[0],V_node[inode].imag[0]]
        i_num_list += [I_node[inode].real[0],I_node[inode].imag[0]]

        V_node_sym_list += [v_real+sym.I*v_imag]
        I_node_sym_list += [i_real+sym.I*i_imag]

        inode += 1
    
    # symbolic voltage and currents vectors (complex)
    V_known_sym = sym.Matrix(V_node_sym_list[:N_v])
    V_unknown_sym = sym.Matrix(V_node_sym_list[N_v:])
    I_known_sym = sym.Matrix(I_node_sym_list[N_v:])
    I_unknown_sym = sym.Matrix(I_node_sym_list[:N_v])
    
    inv_Y_ii_re = inv_Y_ii.real
    inv_Y_ii_im = inv_Y_ii.imag

    inv_Y_ii_re[np.abs(inv_Y_ii_re)<1e-8] = 0
    inv_Y_ii_im[np.abs(inv_Y_ii_im)<1e-8] = 0

    inv_Y_ii = inv_Y_ii_re+sym.I*inv_Y_ii_im

    I_aux = ( I_known_sym - Y_iv @ V_known_sym)  
    g_cplx = -V_unknown_sym + inv_Y_ii @ I_aux

    g_list = []
    for item in g_cplx:
        g_list += [sym.re(item)]
        g_list += [sym.im(item)]

    y_list = v_list[2*N_v:]
    y_0_list = v_num_list[2*N_v:]

    u_dict = dict(zip(v_list_str[:2*N_v],v_num_list[:2*N_v]))
    u_dict.update(dict(zip(i_list_str[2*N_v:],i_num_list[2*N_v:])))

    h_dict = {}
    for bus in load_buses:

        v_a = V_node_sym_list[nodes_list.index(f'{bus}.1')]
        v_b = V_node_sym_list[nodes_list.index(f'{bus}.2')]
        v_c = V_node_sym_list[nodes_list.index(f'{bus}.3')]
        v_n = V_node_sym_list[nodes_list.index(f'{bus}.4')]

        i_a = I_node_sym_list[nodes_list.index(f'{bus}.1')]
        i_b = I_node_sym_list[nodes_list.index(f'{bus}.2')]
        i_c = I_node_sym_list[nodes_list.index(f'{bus}.3')]
        i_n = I_node_sym_list[nodes_list.index(f'{bus}.4')]


        v_an = v_a - v_n
        v_bn = v_b - v_n
        v_cn = v_c - v_n

        s_a = v_an*sym.conjugate(i_a)
        s_b = v_bn*sym.conjugate(i_b)
        s_c = v_cn*sym.conjugate(i_c)

        s = s_a + s_b + s_c
        p_a,p_b,p_c = sym.symbols(f'p_{bus}_a,p_{bus}_b,p_{bus}_c')
        q_a,q_b,q_c = sym.symbols(f'q_{bus}_a,q_{bus}_b,q_{bus}_c')
        g_list += [p_a + sym.re(s_a)]
        g_list += [p_b + sym.re(s_b)]
        g_list += [p_c + sym.re(s_c)]
        g_list += [q_a + sym.im(s_a)]
        g_list += [q_b + sym.im(s_b)]
        g_list += [q_c + sym.im(s_c)]

        g_list += [sym.re(i_a+i_b+i_c+i_n)]
        g_list += [sym.im(i_a+i_b+i_c+i_n)]

        buses_list = [bus['bus'] for bus in grid_1.buses]

        for phase in ['a','b','c']:
            i_real,i_imag = sym.symbols(f'i_{bus}_{phase}_r,i_{bus}_{phase}_i', real=True)
            y_list += [i_real,i_imag]
            i_cplx = I_node[grid_1.nodes.index(f'{bus}.{a2n[phase]}')][0]
            y_0_list += [i_cplx.real,i_cplx.imag]
            u_dict.pop(f'i_{bus}_{phase}_r')
            u_dict.pop(f'i_{bus}_{phase}_i')
            p_value = grid_1.buses[buses_list.index(bus)][f'p_{phase}']
            q_value = grid_1.buses[buses_list.index(bus)][f'q_{phase}']
            u_dict.update({f'p_{bus}_{phase}':p_value})
            u_dict.update({f'q_{bus}_{phase}':q_value})

        
        i_real,i_imag = sym.symbols(f'i_{bus}_n_r,i_{bus}_n_i', real=True)
        y_list += [i_real,i_imag]    
        i_cplx = I_node[grid_1.nodes.index(f'{bus}.{a2n["n"]}')][0]
        y_0_list += [i_cplx.real,i_cplx.imag]
        

        
    return y_list,g_list,y_0_list,u_dict

In [50]:
nodes_list = grid_1.nodes
I_node = grid_1.I_node
V_node = grid_1.V_node
Y_ii = grid_1.Y_ii.toarray()
Y_iv = grid_1.Y_iv
inv_Y_ii = np.linalg.inv(Y_ii)
N_nz_nodes = grid_1.params_pf[0].N_nz_nodes
N_v = grid_1.params_pf[0].N_nodes_v

y_list,g_list,y_0_list,u_dict = grid2dae(nodes_list,V_node,I_node,Y_iv,inv_Y_ii,load_buses=['B2','B3'])

In [51]:
u_dict

{'v_B1_a_r': 11547.0,
 'v_B1_a_i': 0.0,
 'v_B1_b_r': -5773.499999999997,
 'v_B1_b_i': -9999.995337498915,
 'v_B1_c_r': -5773.5000000000055,
 'v_B1_c_i': 9999.99533749891,
 'i_B3_n_r': -2.2737367544323206e-13,
 'i_B3_n_i': 1.1368683772161603e-13,
 'i_B2_n_r': 0.0,
 'i_B2_n_i': 0.0,
 'p_B2_a': 0.0,
 'q_B2_a': -0.0,
 'p_B2_b': -0.0,
 'q_B2_b': 0.0,
 'p_B2_c': 0.0,
 'q_B2_c': 0.0,
 'p_B3_a': -85000.00011979532,
 'q_B3_a': -52678.26838479351,
 'p_B3_b': -85000.00011979532,
 'q_B3_b': -52678.2683847935,
 'p_B3_c': -85000.00011979532,
 'q_B3_c': -52678.26838479348}

In [52]:
N_v

3

In [53]:
x_dummy,u_dummy = sym.symbols('x_dummy,u_dummy')
params = {'a':1}
u_dict.update({'u_dummy':1.0})

sys_dict = {'name':'grid_3bus4wire',
           'params_dict':params,
           'f_list':[(u_dummy-x_dummy)],
           'g_list':g_list,
           'x_list':[x_dummy],
           'y_ini_list':y_list,
           'y_run_list':y_list,
           'u_run_dict':u_dict,
           'u_ini_dict':u_dict,
           'h_dict':{'x_dummy':x_dummy}
           }

db.system(sys_dict)
db.sys2num(sys_dict)